# Compute N<sub>2</sub> fluxes from ERA5 heat fluxes

In [1]:
import os
import yaml
import xarray as xr

import cdsapi

import config

## Get the heat flux

In [ ]:
year0, yearf = 1990, 2021

file_out = f"{config.intake_cachedir}/era5.mean_surface_heat_fluxes.{year0:04d}-{yearf-1:04d}.nc"

if not os.path.exists(file_out):
    
    try:
        with open("ecmwf_credentials.yaml") as fid:
            credentials = yaml.safe_load(fid)
    except:
        raise    
    
    c = cdsapi.Client(**credentials)
    c.retrieve(
        'reanalysis-era5-single-levels-monthly-means',
        {
            'format': 'netcdf',
            'variable': [
                'mean_surface_latent_heat_flux', 
                'mean_surface_net_long_wave_radiation_flux', 
                'mean_surface_net_short_wave_radiation_flux',
                'mean_surface_sensible_heat_flux',
                'sea_surface_temperature',
            ],
            'year': [f"{y:04d}" for y in range(year0, yearf)],
            'month': [f"{m:02d}" for m in range(1, 13)],
            'time': '00:00',
            'product_type': 'monthly_averaged_reanalysis',
        },
        file_out)
    
ds = xr.open_dataset(file_out)
ds    

/glade/work/mclong/miniconda3/envs/sno/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
2021-09-22 12:06:30,262 INFO Welcome to the CDS
2021-09-22 12:06:30,263 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
/glade/work/mclong/miniconda3/envs/sno/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cds.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
2021-09-22 12:06:30,391 INFO Request is queued
/glade/work/mclong/miniconda3/envs/sno/lib/pytho

In [ ]:
for v in ds.data_vars:
    print(f"{v}: {ds[v].attrs['long_name']}")

In [ ]:
dsi = xr.open_dataset("/glade/work/whokim/data/ERA5/ERA5_srfc_fluxes_mean_rate.nc")
for v in dsi.data_vars:
    print(f"{v}: {dsi[v].attrs['long_name']}")